In [255]:
import pandas as pd
import plotly.express as px
from scipy.stats import mannwhitneyu
import plotly.graph_objects as go
import statsmodels.api as sm


In [7]:
# Import CSV files
activities = pd.read_csv('activities.csv')
users = pd.read_csv('users.csv')

In [8]:
# Always show head to confirm data loading went well and to explore the data
activities.head()

,UserId,Date,Revenue
0,1326865,2017-02-19,17.05
1,1326865,2017-02-28,17.05
2,1326865,2017-03-06,17.16
3,1326865,2017-03-07,17.16
4,1326865,2017-03-08,17.16


In [9]:
users.head()

,UserId,Country,DOB,Gender,RegDate
0,1326865,DE,1988-05-04,M,2016-03-24
1,1333278,GB,1971-07-21,M,2016-02-09
2,1336967,GB,1983-07-31,M,2016-01-29
3,1339784,Other,1976-09-21,M,2016-01-21
4,1339829,Other,1981-05-03,F,2016-04-12


# 1
a.	How many variables are in the datasets?

b.	How many observations are in the datasets?

In [10]:
print(activities.shape) # There are 22,642 observations and 3 variables
print(users.shape) # There are 5,807 observations and 5 variables

(22642, 3)
(5807, 5)


# 2
How many:

a.	Male users are in the dataset?

b.	Female users are in the dataset?

c.	For how many users is no gender information available?


In [11]:
users['Gender'].value_counts() # 2,909 males, 1,417 females, 1,481 users for which no gender information is available

M    2909
     1481
F    1417
Name: Gender, dtype: int64

In [13]:
# The assignment states:
# If there is no gender information available, please assume that the gender is male (“M”) for all further questions.

# So I will make sure that is reflected in the dataset, by replacing the empty values with 'M':
users['Gender'].replace(' ', 'M', inplace=True)

# 3 & 4
Merge the two datasets and calculate the days since registration for each activity. What is the total mean and median revenue generated per user (ie across all activities)? How do you interpret the result?

In [16]:
users_and_activities = pd.merge(users, activities, on='UserId') # Merge datasets on UserId column

users_and_activities.head()

,UserId,Country,DOB,Gender,RegDate,Date,Revenue
0,1326865,DE,1988-05-04,M,2016-03-24,2017-02-19,17.05
1,1326865,DE,1988-05-04,M,2016-03-24,2017-02-28,17.05
2,1326865,DE,1988-05-04,M,2016-03-24,2017-03-06,17.16
3,1326865,DE,1988-05-04,M,2016-03-24,2017-03-07,17.16
4,1326865,DE,1988-05-04,M,2016-03-24,2017-03-08,17.16


In [59]:
# Note that every single row is one activity, multiple rows can belong to one user. Therefore we first have to group:
total_revenue = users_and_activities.groupby('UserId')['Revenue'].sum() # Total revenue per user across all activities

print("Mean revenue:", total_revenue.mean())
print("Median revenue:", total_revenue.median())

# Given the fact that the mean revenue is way higher than the median (131 and 40),
# This means that the distribution is right-skewed.
# This implies that there are a lot of users that generated relatively low amounts,
# And a few users that generated relatively high amounts, meaning the mean ends up way higher than the median value

# We can confirm this by plotting a histogram of the data:
fig = px.histogram(total_revenue)
fig.show()
# The histogram below indeed confirms our initial thought, the distribution is severly right-skewed
# (Zoomed-in to make it even more clear)

Mean revenue: 512.6906141107444
Median revenue: 80.0


# 5
What is the average week 1 revenue, ie the revenue generated by the user in their first week (ie within the first 7 days since registration)?

In [74]:
# Date columns contain the data type string, convert them to datetime first so we can calculate date differences
users_and_activities['Date'] = pd.to_datetime(users_and_activities['Date'])
users_and_activities['RegDate'] = pd.to_datetime(users_and_activities['RegDate'])

# Subtract one from the other and save in new column
users_and_activities['DaysSinceReg'] = users_and_activities['Date'] - users_and_activities['RegDate']

# Filter dataframe for activities within the first week
users_and_activities_firstweekonly = users_and_activities[users_and_activities['DaysSinceReg'] <= pd.Timedelta(7,'D')]

# Calculate average 1 week revenue
users_total_first_week_revenue = users_and_activities_firstweekonly.groupby('UserId')['Revenue'].sum()
print("The average 1 week revenue is:", round(users_total_first_week_revenue.mean(),2))

The average 1 week revenue is: 200.25


# 6
In this dataset men generated a higher week 1 revenue than women, on average.  Is this difference in revenue between men and women statistically significant? What is an appropriate statistical test to determine this and what is it’s p-value?

In [195]:
# Let's first put everything in a neat dataframe:
users_total_first_week_revenue.rename("TotalRevenue", inplace=True)
users_and_activities_with_totals = pd.merge(users_total_first_week_revenue, users_and_activities_firstweekonly, on='UserId')
users_and_activities_1week_revenue = users_and_activities_with_totals.drop_duplicates(subset=['UserId'])
users_and_activities_1week_revenue.head()

,UserId,TotalRevenue,Country,DOB,Gender,RegDate,Date,Revenue,DaysSinceReg
0,3092167,25.00,GB,1987-02-08,M,2016-10-21,2016-10-28,25.00,7 days
1,3095357,200.00,GB,1972-06-09,M,2016-10-30,2016-11-06,200.00,7 days
2,3109386,20.00,GB,1986-05-16,F,2017-02-26,2017-03-03,20.00,5 days
3,3114216,29.19,Other,1978-09-02,M,2017-01-23,2017-01-29,29.19,6 days
4,3114891,20.00,GB,1968-03-23,M,2017-01-16,2017-01-19,20.00,3 days


In [196]:
# A t-test could be appropriate if the means of the different samples are normally distributed.
# which is probably not the case for revenue as we have concluded across all genders in the previous questions.
# Let's check this again but now for each gender:

fig = px.histogram(users_and_activities_1week_revenue[users_and_activities_1week_revenue['Gender'] == 'M']['TotalRevenue'],
                  title='Total week 1 revenue across all activites for male users')
fig.show()

fig = px.histogram(users_and_activities_1week_revenue[users_and_activities_1week_revenue['Gender'] == 'F']['TotalRevenue'],
                  title='Total week 1 revenue across all activites for female users')
fig.show()

# Both histograms are indeed heavily right-skewed (zoomed-in to make it clear), so not normally distributed.
# Therefore we will apply a non-parametric statistical test to compare the revenues between genders

In [197]:
# For the reasons described in the cell above, we will use a nonparametric test.
# We have two independent samples (males and females), and want to test whether the difference in revenue is different
# Thus, we use the Mann-Whitney test to determine if the samples come from a single population or from two different populations

# We want to test whether 1 week revenue for males is higher than for females, therefore we will do a one-tailed test
# The null hypothesis can then be defined as: week 1 revenue is the same for male and female users
# The alternative hypothesis: week 1 revenue is higher for male users

mannwhitneyu(
                users_and_activities_1week_revenue[users_and_activities_1week_revenue['Gender'] == 'M']['TotalRevenue'],
                users_and_activities_1week_revenue[users_and_activities_1week_revenue['Gender'] == 'F']['TotalRevenue'],
                alternative='greater'
            )

# The p-value is 0.002, meaning that we reject the null hypothesis that the 1 week revenue is the same for males and females,
# We accept the alternative hypothesis that the 1 week revenue is higher for males than for females

MannwhitneyuResult(statistic=2451993.0, pvalue=0.001979181679232299)

# 7
In which country is this difference biggest?

In [198]:
# This is a tricky questions, since we could simply calculate the mean 1 week revenue per country for both males and females
# and check where the difference is biggest, however with some countries having more data points than others, this could
# give the wrong idea about the data. For example, if one country has only a couple entries that happen to be far apart
# for males and females, the absolute difference might be biggest here, but not as statistically significant as
# differences in other countries.
# So for this question, I will evaluate in which country the difference is most significant.
# The differences in means are shown as well, but in question 8

unique_countries = users_and_activities_1week_revenue['Country'].unique() # First get all the unique countries

stat_test_all_countries = {}

# Then perform the statistical test for every country:
for country in unique_countries:
    users_from_this_country = users_and_activities_1week_revenue[users_and_activities_1week_revenue['Country'] == country]
    
    p_value = mannwhitneyu(
                                users_from_this_country[users_from_this_country['Gender'] == 'M']['TotalRevenue'],
                                users_from_this_country[users_from_this_country['Gender'] == 'F']['TotalRevenue'],
                                alternative='greater'
                          ).pvalue
    
    stat_test_all_countries[country] = p_value

print("The biggest difference in 1 week revenue between males and females is in:",
      min(stat_test_all_countries, key=stat_test_all_countries.get))

The biggest difference in 1 week revenue between males and females is in: GB


# 8
Please visualise the relationship between country, gender and week 1 revenue with an appropriate chart.

In [199]:
male_1week_revenue = users_and_activities_1week_revenue[
    users_and_activities_1week_revenue['Gender'] == 'M'].groupby('Country')['TotalRevenue'].mean()

female_1week_revenue = users_and_activities_1week_revenue[
    users_and_activities_1week_revenue['Gender'] == 'F'].groupby('Country')['TotalRevenue'].mean()

fig = go.Figure(data=[
    go.Bar(name='Average 1 week revenue for males', x=male_1week_revenue.index, y=male_1week_revenue),
    go.Bar(name='Average 1 week revenue for females', x=female_1week_revenue.index, y=female_1week_revenue)
])
# Change the bar mode
fig.update_layout(barmode='group', title='1 week revenue per country for males and females')
fig.show()

# 9
Build a linear regression model to predict the week 1 revenue of a user based on the following variables: Gender, Age, Country and day 1 revenue (ie the revenue generated by the user on their registration date). Which of these variables have a statistically significant predictive power according to this model?

In [212]:
# Let's add the day 1 revenue to the dataframe first
users_and_activities_firstdayonly = users_and_activities[users_and_activities['DaysSinceReg'] <= pd.Timedelta(1,'D')]
users_total_first_day_revenue = users_and_activities_firstdayonly.groupby('UserId')['Revenue'].sum()
users_total_first_day_revenue.rename("FirstDayRevenue", inplace=True)
print(users_total_first_day_revenue.shape)
print(users_total_first_week_revenue.shape)

# As we see here, not all users have revenue in the dataset for the first day, since the shapes do not match:
# It is a small difference, in this case I choose to drop the users which do not have both a 1 week and 1 day revenue
users_and_activities_q9 = pd.merge(users_and_activities_1week_revenue, users_total_first_day_revenue, on='UserId')

(4960,)
(5105,)


In [248]:
# There is no age column yet so we have to add it
users_and_activities_q9['DOB'] = pd.to_datetime(users_and_activities_q9['DOB'])
users_and_activities_q9['Age'] = round((pd.to_datetime("today") - users_and_activities_q9['DOB']).dt.days / 365.25)
users_and_activities_q9.head()

,UserId,TotalRevenue,Country,DOB,Gender,RegDate,Date,Revenue,DaysSinceReg,FirstDayRevenue,Age
0,3131151,44.54,DE,1990-04-24,M,2016-10-22,2016-10-23,44.54,1 days,44.54,32.0
1,3132299,25.58,US,1984-03-23,M,2017-01-05,2017-01-06,25.58,1 days,25.58,38.0
2,3133634,42.63,US,1966-01-19,M,2017-01-14,2017-01-14,42.63,0 days,42.63,56.0
3,3133639,17.05,Other,1990-04-01,M,2016-12-02,2016-12-03,17.05,1 days,17.05,32.0
4,3133644,20.00,GB,1989-08-05,F,2016-09-26,2016-09-26,20.00,0 days,20.00,32.0


In [280]:
X = users_and_activities_q9[["Gender", "Age", "Country", "FirstDayRevenue"]]  # values converts it into a numpy array
X = pd.get_dummies(data=X, drop_first=False)
Y = users_and_activities_q9['TotalRevenue']
mod = sm.OLS(Y,X)
fii = mod.fit()
p_values = fii.summary2().tables[1]['P>|t|']

print(p_values) # Only FirstDayRevenue has a statistically predictive power according to this model

Age                0.118682
FirstDayRevenue    0.000000
Gender_F           0.346741
Gender_M           0.327866
Country_DE         0.302896
Country_FR         0.936697
Country_GB         0.969338
Country_Other      0.677672
Country_US         0.418291
Name: P>|t|, dtype: float64


# 10
What revenues do you expect women from France, Germany and the UK to generate, assuming they are all aged 40 and all generated £20 on their registration day?

In [290]:
print(f"Woman (DE) (40 yo) that generated 20 on their registration day is expected to generate {round(fii.predict([40,20,1,0,1,0,0,0,0])[0],1)} in 1 week")
print(f"Woman (FR) (40 yo) that generated 20 on their registration day is expected to generate {round(fii.predict([40,20,1,0,0,1,0,0,0])[0],1)} in 1 week")
print(f"Woman (UK) (40 yo) that generated 20 on their registration day is expected to generate {round(fii.predict([40,20,1,0,0,0,1,0,0])[0],1)} in 1 week")

Woman (DE) (40 yo) that generated 20 on their registration day is expected to generate 24.1 in 1 week
Woman (FR) (40 yo) that generated 20 on their registration day is expected to generate 41.1 in 1 week
Woman (UK) (40 yo) that generated 20 on their registration day is expected to generate 42.5 in 1 week
